In [5]:
! pip install openpyxl

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 20.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [29]:
import pandas as pd

channels = pd.read_excel("../data/tg_channels.xlsx")
df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [30]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df.head()

,message_id,id_channel,message,date,topic,channel_name
0,0007e2f8-787d-404f-91ff-e2582096a4a7,18,Сербия согласна поддержать санкции Евросоюза п...,2025-07-26 07:01:09,Санкции и геополитика,Экономика
1,000884a5-8291-4ec1-805f-ac131112aaf7,6,Китайский рынок акций упал сильнее всего с апр...,2025-09-04 10:16:56,Рынки капитала,Forbes Russia
2,000b0331-92a9-4eb4-9f58-d00811257758,18,Министерство труда США отменило рекомендации 2...,2025-05-29 04:05:09,Государственная экономическая политика,Экономика
3,000b8df7-d902-41eb-b668-900614902f0a,6,Чистая прибыль Московской биржи по МСФО во вто...,2025-08-26 11:40:55,Корпоративные финансы,Forbes Russia
4,0011adea-7a98-4dcc-b753-905597b42788,4,"США хотят получить нефть и «всё, что угодно» о...",2025-02-22 20:57:18,Сырьевые рынки,"Банки, деньги, два офшора"


In [31]:
import re
import numpy as np
import pandas as pd

def clean_news_text(t: str) -> str:
    t = t or ""
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [65]:
df.head()

,message_id,id_channel,message,date,topic,channel_name,date_day
0,0007e2f8-787d-404f-91ff-e2582096a4a7,18,Сербия согласна поддержать санкции Евросоюза п...,2025-07-26 07:01:09+00:00,Санкции и геополитика,Экономика,2025-07-26 00:00:00+00:00
1,000884a5-8291-4ec1-805f-ac131112aaf7,6,Китайский рынок акций упал сильнее всего с апр...,2025-09-04 10:16:56+00:00,Рынки капитала,Forbes Russia,2025-09-04 00:00:00+00:00
2,000b0331-92a9-4eb4-9f58-d00811257758,18,Министерство труда США отменило рекомендации 2...,2025-05-29 04:05:09+00:00,Государственная экономическая политика,Экономика,2025-05-29 00:00:00+00:00
3,000b8df7-d902-41eb-b668-900614902f0a,6,Чистая прибыль Московской биржи по МСФО во вто...,2025-08-26 11:40:55+00:00,Корпоративные финансы,Forbes Russia,2025-08-26 00:00:00+00:00
4,0011adea-7a98-4dcc-b753-905597b42788,4,"США хотят получить нефть и «всё, что угодно» о...",2025-02-22 20:57:18+00:00,Сырьевые рынки,"Банки, деньги, два офшора",2025-02-22 00:00:00+00:00


In [33]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [38]:
! pip install faiss

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [39]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

E_docs = encoder.encode(
    doc_inputs,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True,
).astype(np.float32)

Batches:   0%|          | 0/291 [00:00<?, ?it/s]

In [41]:
! pip install faiss-cpu faiss-gpu-cu12 faiss-gpu-cu11

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 116.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 115.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 108.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 161.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 16.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fsclient 1.1.2 requires numpy<2.0.0, but you hav

In [42]:
import faiss

dim = E_docs.shape[1]
index = faiss.IndexFlatIP(dim)      
index.add(E_docs)

In [43]:
# save
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

OUT = Path("indexes")
OUT.mkdir(parents=True, exist_ok=True)

#rowmap
rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()
rowmap.to_parquet(OUT / "rowmap.parquet", index=False)

np.save(OUT / "E_docs_e5_base.npy", E_docs)

faiss.write_index(index, str(OUT / "faiss_e5_base.index"))

with open(OUT / "bm25_corpus_tok.pkl", "wb") as f:
    pickle.dump(corpus_tok, f)


In [44]:
# load
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

INP = Path("indexes")

rowmap = pd.read_parquet(INP / "rowmap.parquet")

E_docs = np.load(INP / "E_docs_e5_base.npy")
index = faiss.read_index(str(INP / "faiss_e5_base.index"))

with open(INP / "bm25_corpus_tok.pkl", "rb") as f:
    corpus_tok = pickle.load(f)

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tok)


In [45]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [12]:
TOPK_DOCS = 30          # сколько документов реально класть в контекст LLM (50 часто слишком много по токенам)
SNIP_CHARS = 900        # длина сниппета одного документа
TOPN_EACH = 1000        # кандидаты для hybrid (RRF)
K_RRF = 60
W_SEM = 1.0
W_BM = 1.0

In [46]:
import numpy as np
import pandas as pd
import re

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k-1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(["query: " + query], normalize_embeddings=True, show_progress_bar=False).astype(np.float32)
    scores, idx = index.search(qv, topN) 
    return idx[0].astype(int), scores[0].astype(np.float32)

def bm25_candidates(bm25, tokenize_fn, query: str, topN: int = 500):
    scores = bm25.get_scores(tokenize_fn(query))
    idx = _topk_indices_from_scores(scores, topN)
    return idx, scores[idx].astype(np.float32)

def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    exclude_message_id: str | None = None,
) -> pd.DataFrame:
    #dense
    d_idx, d_sc = dense_candidates_faiss(index, encoder, query, topN=topN_each)
    dense_rank = {int(i): r for r, i in enumerate(d_idx, start=1)}

    #bm25
    b_idx, b_sc = bm25_candidates(bm25, tokenize_fn, query, topN=topN_each)
    bm_rank = {int(i): r for r, i in enumerate(b_idx, start=1)}

    #union+RRF
    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    rrf = np.zeros(len(union), dtype=np.float32)

    for j, i in enumerate(union):
        if i in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[i])
        if i in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[i])

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]

    out = df.iloc[union].copy()
    out["score_rrf"] = rrf

    out["rank_dense"] = out.index.map(lambda ridx: dense_rank.get(int(ridx), np.nan))
    out["rank_bm25"]  = out.index.map(lambda ridx: bm_rank.get(int(ridx), np.nan))

    if exclude_message_id is not None:
        out = out[out["message_id"].astype(str) != str(exclude_message_id)]

    return out.head(k).reset_index(drop=True)


In [105]:
RAG_SUMMARY_SYSTEM = """Ты — аналитик экономических новостей.

ВАЖНО:
- Отвечай СТРОГО на русском языке.
- Используй ТОЛЬКО информацию из предоставленных документов.
- Не добавляй фактов, которых нет в источниках. Если чего-то не хватает — прямо пиши, что это неизвестно.
- Не вставляй служебные фразы/мусор (например "水平滚动以查看内容...").
- Обязательно ставь ссылки на источники в формате [1], [2] ... по номеру документа.
- Перепечатки и повторы объединяй: одинаковые факты не повторяй, а группируй.

Задача:
По запросу пользователя подготовь развернутый отчёт о развитии сюжета. Упор на хронологию, динамику обсуждения и различия по каналам.

Вход:
Ниже будут документы вида:
[i] date=YYYY-MM-DD channel=<название> id=<message_id>
<текст>

Сформируй ответ в следующем формате (строго соблюдай заголовки):

# 1) Короткое саммари (5–8 предложений)
Кратко объясни, что происходит по теме запроса: ключевой факт/триггер, почему это важно, какая общая динамика.

# 2) Хронология (таймлайн)
Сортируй по датам от раннего к позднему.
Для каждой даты:
- YYYY-MM-DD — 1–3 пункта, каждый пункт = один конкретный факт/изменение/заявление + ссылки [..]
Если в один день много однотипных перепечаток — сожми до 1 пункта и укажи, что это широко тиражировалось.

# 3) Кто и о чём пишет (по каналам)
Сгруппируй по каналам.
Для каждого канала:
- <channel>: что именно добавляет/подчеркивает (1–3 предложения) + ссылки [..]
Если канал просто перепечатывает — так и напиши.

# 4) Насколько “горячая” новость сейчас
Оцени по датам публикаций в источниках:
- Последняя дата в документах: <YYYY-MM-DD>
- “Пик обсуждения”: какие даты встречаются чаще всего (можно 1–3 даты) + ссылки на примеры [..]
- Вывод: горячая / умеренно актуальная / устаревшая
Критерий: чем ближе последние публикации и чем плотнее события по дням, тем горячее. Если в источниках нет явной плотности — скажи это.

# 5) Итоговый вывод (эссе-абзац)
Сделай 1–2 абзаца в стиле аналитического отчёта:
- что это значит для экономики/рынков (только если есть в источниках),
- как менялся нарратив/акценты по мере развития,
- какие есть неопределённости или развилки.

# 6) Неопределённости и что нужно проверить
Список 3–6 пунктов: чего не хватает в источниках, что осталось неясным, какие факты требуют подтверждения.
"""




def build_rag_context(query: str, cand: pd.DataFrame, k_docs: int = 25, snip_chars: int = 900) -> str:
    c = cand.head(k_docs).copy()

    blocks = []
    for i, row in enumerate(c.itertuples(index=False), start=1):

        date = getattr(row, "date", "")
        channel  = getattr(row, "channel_name", getattr(row, "id_channel", ""))
        text     = getattr(row, "message_clean", getattr(row, "message", ""))

        meta = f"Источник [{i}] | date={date} | channel={channel}"
        blocks.append(meta + "\n" + snippet(str(text), snip_chars))

    docs_txt = "\n\n".join(blocks)
    return f"ЗАПРОС:\n{query}\n\nДОКУМЕНТЫ:\n{docs_txt}"


In [95]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (ro

In [131]:
import torch

@torch.inference_mode()
def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame,
                  k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000) -> str:
    user = build_rag_context(query, cand, k_docs=k_docs, snip_chars=snip_chars)

    messages = [
        {"role": "system", "content": RAG_SUMMARY_SYSTEM},
        {"role": "user", "content": user},
    ]
    prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    enc = sum_tokenizer(prompt, return_tensors="pt", truncation=True).to(sum_model.device)

    out_ids = sum_model.generate(
        **enc,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=sum_tokenizer.eos_token_id,
        pad_token_id=sum_tokenizer.eos_token_id,
    )
    prompt_len = int(enc["attention_mask"][0].sum().item())
    return sum_tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip()

In [133]:
def run_rag_hybrid(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    exclude_message_id: str | None = None,
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000, 

    sum_model=None,
    sum_tokenizer=None,
):
    cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_fn,
        query=query,
        k=k_retrieve,
        topN_each=topN_each,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        exclude_message_id=exclude_message_id)

    ctx = build_rag_context(query, cand, k_docs=k_docs, snip_chars=snip_chars)
    
    if sum_model is None or sum_tokenizer is None:
        return {"context": ctx, "candidates": cand}
    
    summary = rag_summarize(sum_model, sum_tokenizer, query, cand, k_docs=k_docs, snip_chars=snip_chars, max_new_tokens=max_new_tokens)
    return {"context": ctx, "summary": summary, "candidates": cand}


In [135]:
q = "Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика."

out = run_rag_hybrid(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    query=q,
    k_retrieve=50,
    topN_each=2000,
    k_docs=30,
    snip_chars=1500,
    max_new_tokens = 2000,
    sum_model=model,           
    sum_tokenizer=tokenizer,
)


In [136]:
display(out["context"])

'ЗАПРОС:\nКофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика.\n\nДОКУМЕНТЫ:\nИсточник [1] | date=2025-04-07 13:32:23+00:00 | channel=Блумберг\nКофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика. 🔵 Bloomberg\n\nИсточник [2] | date=2025-03-24 06:48:20+00:00 | channel=Банки, деньги, два офшора\nКофе может резко подорожать до 40% уже в этом году, предупреждают участники рынка. Сильнее всего подорожают сорта арабика и органический кофе. Это произойдёт из-за засухи в Бразилии и роста логистических издержек. @bankrollo\n\nИсточник [3] | date=2025-03-24 06:06:46+00:00 | channel=Блумберг\nРоссиян предупредили о резком скачке цен на

In [137]:
out["summary"]

'# 1) Короткое саммари\nКофе в России может подорожать на 50% уже в этом году из-за проблем с урожаем в Бразилии и Вьетнаме, а также роста логистических расходов. Аналитики предупреждают о значительном удорожании сортов арабики и органического кофе. Несмотря на некоторые колебания в прогнозах, общая динамика свидетельствует о значительном росте цен на кофе в течение 2025 года.\n\n# 2) Хронология (таймлайн)\n- 2025-01-09: Цены на кофе и шоколад могут подорожать на 25% в этом году, предупреждают эксперты. [4]\n- 2025-01-13: Цены на стройматериалы в России могут взлететь на 30% уже в первом полугодии. [21]\n- 2025-01-14: Бензин в России может подорожать на 5–8,5% в 2025 году из-за роста акцизов и санкций. [25]\n- 2025-01-27: Биржевые цены на арабику достигли нового рекорда – более $7,8 тыс за тонну. [23]\n- 2025-01-29: Цены на кофе в России подскочат ещё на 20% уже весной. [9]\n- 2025-03-04: Кофе рекордно подорожает на 30-40% уже к концу года, пишет Газета.ру. [16]\n- 2025-03-17: Аналитик